# LTAT.02.006 Andmeteaduse meetodid, sügis 2022
# 3. kodutöö

## Tähtaeg: esmaspäev, 17. oktoober kell 12:00 (keskpäeval)

### Kodutöö reeglid

Palun kontrollige üle kodutööde reeglid esimesest kodutööst, samad reeglid kehtivad siin.

### Palun lugege kokku kodutööle kulunud aeg

Palun, et loeksite kokku kodutööle kulunud aeg, eriti hea kui ka eraldi iga ülesande peale kulunud aeg. Eraldi ülesannete alapunktide kaupa pole aega vaja eristada. Kulunud aeg kirjutage palun selleks mõeldud lahtritesse kodutöö lõpus. See ei ole kohustuslik ega mõjuta hinnet, kuid aitab kodutöid paremaks teha.

# 1. ülesanne.  Kliimamudelid Tartu kohta - KDE ja ristvalideerimine (4,5 punkti)

Failis `kliima_tartu.csv` on andmed, mis on laetud alla lehelt `https://www.ncdc.noaa.gov/cag/global/time-series/58.4,26.7` (otselink failile: `https://www.ncdc.noaa.gov/cag/global/time-series/58.4,26.7/land_ocean/all/8/1880-2020/data.csv`). Nendes andmetes on toodud Tartu õhutemperatuuri kuukeskmiste erinevus pikaajalisest vastava kuu keskmisest. Seejuures on pikaajaline keskmine arvutatud üle aastate 1981-2010 (niinimetatud baasperiood ehk ingl.k. 'base period'). Näiteks faili esimene andmerida sisaldab arve `188001,-2.88`, mis tähendab, et kuu 1880/01 (ehk jaanuar 1880) keskmine oli baasaastate 1981-2010 kuukeskmisest 2.88 kraadi võrra madalam. Neid arve, mis mõõdavad kuukeskmise erinevust pikaajalisest kekmisest, nimetame ka anomaaliateks. Näiteks kuu 1880/01 anomaalia oli -2.88. Keda huvitab, siis täpsem selgitus andmete kohta on toodud lehel `https://www.ncdc.noaa.gov/monitoring-references/faq/anomalies.php`. 

Kodutöö esimeses ülesandes modelleerime anomaaliate jaotust meetodiga KDE. Uurime 2-osalise ristvalideerimise abil, et milline on parim väärtus KDE ribalaiuse parameetrile (ingl.k. 'bandwidth'). Seejuures vaatleme millistes ribalaiuse vahemikes toimub ala- ja üle-sobitamine andmetele.

Alustame andmete sisselugemisega. Kuna andmefailis on kõigepealt 4 tekstirida enne kui andmed pihta hakkavad, siis jätame need read välja

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats

d = pd.read_csv('kliima_tartu.csv',header=4)
d

,Year,Value
0,188001,-2.88
1,188002,-1.21
2,188003,-3.16
3,188004,-2.07
4,188005,-1.87
...,...,...
1683,202004,0.05
1684,202005,-1.26
1685,202006,3.11
1686,202007,-0.83


Järgnevalt eraldame 6-kohalisest kuu-numbrist aasta ja kuu ning anname veergudele eestikeelsed nimed.

In [ ]:
# eraldame 
d['aasta'] = d['Year'] // 100
d['kuu'] = d['Year'] % 100
d['anomaalia'] = d['Value']
d = d.drop(columns=['Year','Value'])
d

,aasta,kuu,anomaalia
0,1880,1,-2.88
1,1880,2,-1.21
2,1880,3,-3.16
3,1880,4,-2.07
4,1880,5,-1.87
5,1880,6,-0.90
6,1880,7,-0.83
7,1880,8,0.39
8,1880,9,-0.04
9,1880,10,-4.94


<font color='purple'>**(a) Leia andmetest kõige äärmuslikumad anomaaliad (kõige positiivsem ja kõige negatiivsem anomaalia). Millal oli pikaajalisest keskmisest kõige rohkem soojem kuu vaadeldud 140-aastases ajaloos? Millal oli pikaajalisest keskmisest kõige rohkem külmem kuu?**
    
</font>

<font color='purple'>**OODATUD VASTUS: Aastaarv, kuu ja anomaalia väärtus kõige positiivsema ja kõige negatiivsema anomaaliaga kuule.** 
        
VIHJE: <br>
Uurige dokumentatsioonist, mida teeb funktsioon `np.argmax(d['anomaalia'])` ja analoogiliselt `np.argmin(d['anomaalia'])`.
</font> 

In [ ]:
# Lahendus palun siia (siin ja edaspidi võib selle kommentaari kustutada 
# ning samuti võib ka ise tekstivälju ja koodivälju juurde teha, kuid 
# muid etteantud välju kustutada ei tohi).

<font color='purple'>**(b) Defineerige funktsioon `def aastafilter(andmed,algusaasta,lõppaasta):`, mis väljastab andmetest vaid need read, millel aasta on `algusaasta` ja `lõppaasta` vahel, kus algusaasta ja lõppaasta on mõlemad kaasa arvatud. Näiteks `aastafilter(d,2000,2002)` peaks väljastama andmetest kõik read, kus aastaks on 2000, 2001 või 2002.**  </font>
    
<font color='purple'>**Arvutage keskmine jaanuari anomaalia üle kõigi baasperioodi aastate ehk üle 1981-2010 (siin ja edaspidi on ajavahemikes nii algusaasta kui lõppaasta kaasa arvatud). See peaks tulema väga väike arv (alla 0.01), sest selle ajavahemiku suhtes arvutataksegi anomaaliaid. Samuti peaks keskmine veebruari anomaalia üle 1981-2010 olema pisike. Kõik kuudekaupa keskmised anomaaliad üle 1981-2010 saate arvutada nii: `aastafilter(d,1981,2010).groupby('kuu')['anomaalia'].mean()`.** </font>
    
<font color='purple'>**Kuna teises ülesandes tahame uurida ka kliimamuutusi, siis tahaksime baasperioodi ümber muuta ja võtta baasperioodiks hoopis saja-aastase ajaperioodi 1880-1979. Selleks arvutage ajavahemiku 1880-1979 jaanuarikuine keskmine anomaalia ning seejärel lahutage saadud väärtus maha kõigist jaanuarikuistest anomaaliatest. Sarnaselt arvutage ajavahemiku 1880-1979 veebruarikuine keskmine anomaalia ning seejärel lahutage saadud väärtus maha kõigist veebruarikuistest anomaaliatest. Ja nõnda tehke kõigi kuude jaoks. Näiteks jaanuarikuu puhul saab sellise lahutamise teha nii: `d.loc[d.kuu==1,'anomaalia_1880_1979'] = d.loc[d.kuu==1,'anomaalia'] - keskmine`, kus `keskmine` on ajavahemiku 1880-1979 jaanuarikuine keskmine anomaalia. Soovitan see `keskmine` ümardada sajandikeni (`keskmine = np.round(keskmine,2)`), kuna andmetes on kõik anomaaliad mõõdetud sajandikes.** </font>
    
<font color='purple'>**Et kontrollida, kas kõik sai õigesti, arvutage nihutatud andmetes kuudekaupa keskmised anomaaliad üle 1880-1979 ning peaksite saama kõik väga väiksed arvud (alla 0.01). Väljastage lõpptulemusena aasta 2020 anomaaliad - kui kõik õigesti tegite, siis need peaksid saja-aastase baasperioodi suhtes tulema ligikaudu +8.16 (jaanuar), +7.42 (veebruar), +5.41 (märts), +1.76 (aprill), -0.17 (mai) jne. Näiteks mai 2020 oli seega baasperioodi keskmisest külmem!** </font>

<font color='purple'> **OODATUD TULEMUS: Keskmine jaanuari anomaalia üle aastate 1981-2010. Aasta 2020 kuude anomaaliad 100-aastase baasperioodi 1880-1979 suhtes.** </font>
    
 <font color='purple'>VIHJE:</font> <br>
<font color='purple'>Funktsiooni `aastafilter` defineerimiseks uurige dokumentatsioonist, mida teeb funktsioon `np.isin(andmed.aasta,range(algusaasta,lõppaasta+1))`. Sellega sama tulemuse annaks `(andmed.aasta>=algusaasta)&(andmed.aasta<=lõppaasta)`. Seejärel peate veel andmetest vastavad read välja võtma: `andmed[...]`.
 </font>

In [ ]:
# Lahendus

<font color='purple'>**(c) Eraldage saja aasta 1880-1979 andmed: `d100 = aastafilter(d,1880,1979)`. Joonistage anomaaliate histogramm üle kõigi kuude andmestikus d100 ning leidke sellele KDE-meetodiga 7 erinevat tihedusfunktsiooni hinnangut, kasutades vastavalt ribalaiuse (`bandwidth`) parameetri väärtuseid 0.02, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5. Selleks, et joonistada histogrammi ja KDE hinnangut samale joonisele, kasutage funktsiooni `plt.hist(d100.anomaalia,density=True,bins=100,alpha=0.5)` (kus `density=True` valib y-telje skaala selliselt, et tulpade pindala väljendab nende proportsionaalset sagedust ehk kasutab tihedusfunktsiooni skaalat, ja `alpha=0.5` teeb tulbad pisut läbipaistvamaks) ning seejärel funktsiooni `plt.plot(x_vektor,kde.pdf(x_vektor),'-')`, kus `x_vektor` on ühtlaste vahedega väärtused andmete miinimumi ja maksimumi vahel (vt. funktsiooni `np.linspace`), ja kus näiteks `kde = scipy.stats.gaussian_kde(d100.anomaalia,bw_method=1)` kasutab ribalaiust 1. Valige välja ribalaiuse parameeter, mis teile silma järgi kõige paremini näib sobivat (ilma mingeid lisaarvutusi tegemata).**</font>
    
<font color='purple'>**OODATUD TULEMUS: Välja valitud ribalaiuse parameetri väärtus ning sellele vastav joonis, kus on histogramm ning selle peal vastava ribalaiusega KDE tihedushinnang.**</font>

In [ ]:
# Lahendus

<font color='purple'>**(d) Järjestage andmestik d100 juhuslikult ümber ning jagage juhuslikult kolmeks võrdse suurusega andmestikuks: treeningandmestik, valideerimisandmestik ning testandmestik (kood selleks on allpool antud, palun ärge seda muutke). Sobitage ainult treeningandmestikul 96 erinevat KDE tihedushinnangut, vastavalt ribalaiuse parameetriga 0.05, 0.06, 0.07, ..., 0.99, 1.00. Selleks, et nende tihedushinnangute headust mõõta ja võrrelda, kasutame mõõtu NLL (average negative log-likelihood) ehk keskmist negatiivset log-tõepära. Järgnevas lõigus on selgitus, mida see tähendab ja kuidas seda arvutada. Põhimõtteliselt on idee selles, et valime ribalaiuse parameetri selliselt, et andmed oleks KDE tihedushinnangu järgi võimalikult suure kogutõenäosusega, ehk negatiivne log-tõepära oleks võimalikult väike.**
</font>

<font color='purple'>**Tõepära on andmepunktide tiheduste korrutis. Olgu meil KDE mudel `kde` ja andmepunktid `x1`, `x2`, `x3`. Siis tõepära on `kde.pdf(x1)*kde.pdf(x2)*kde.pdf(x3)`. Kuna paljude väikeste arvude korrutis läheb kiiresti nulli, siis kasutatakse praktikas log-tõepära, ehk `np.log(kde.pdf(x1))+np.log(kde.pdf(x2))+np.log(kde.pdf(x3))`. Kuna see on tüüpiliselt negatiivne suurus, siis muudetakse sellel tavaliselt märk ning lisaks kasutatakse tavaliselt summa asemel keskmist. Mida väiksem on negatiivne log-tõepära, seda parem on tihedushinnang. Kui andmed on vektoris `x_vektor`, siis keskmist negatiivset log-tõepära saab arvutada nii: `np.mean(-np.log(kde.pdf(x_vektor)))`.**</font>
    
<font color='purple'>**Arvutage NLL iga erineva ribalaiuse jaoks ning eraldi treeningandmestikul ning valideerimisandmestikul (kokku seega 96 ribalaiust korda 2 andmestikku ehk 192 NLL väärtust). Tehke joonis, kus x-teljel on ribalaius ja y-teljel NLL ning NLL on visualiseeritud kahe eraldi joonena - üks treening- ning teine valideerimisandmestikul. Tehke joonte kohta ka legend - selleks lisage joont defineerivale `plt.plot` käsule argumendina juurde `label='NLL treeningandmetel'` või `label='NLL valideerimisandmetel'` ning peale seda lisage käsk `plt.legend()`. Kirjeldage, millises ribalaiuse piirkonnas toimub alasobitamine ja millises ülesobitamine. Millise väärtuse valiksite ribalaiusele ning mis on treening- ja valideerimis-NLL selle ribalaiuse jaoks?**</font>
    
<font color='purple'>**OODATUD TULEMUS: Joonis, mis näitab NLL mõõtu treening- ning valideerimisandmetel erinevate ribalaiustega. Selgitus, kus on ala- ja ülesobitamise piirkonnad (1-3 lauset). Selgitus, millise ribalaiuse võiks valida ning mis on selle treening- ja valideerimis-NLL (1-3 lauset).**</font>

<font color='purple'>VIHJED:</font> <br> 

<font color='purple'>Ala- ja ülesobitamise piirkondi leides tasuks ehk vaadata korraks ka eelmises alapunktis tehtud erinevate ribalaiuste jooniseid vaadata ning mõelda sellele, et kas ribalaiuse kasvades mudeli keerukus kasvab või kahaneb.
</font>

In [ ]:
# siin kodutöös kasutame erandina aegunud juhuslikkuse seemet
np.random.seed(0)
### Palun seda koodi mitte muuta. (Välja arvatud "#"-de eemaldamine.)
### Palun eemaldage kommentaarimärgid järgmistelt ridadelt (praegu on välja kommenteeritud, sest kui te pole veel d100 defineerinud, siis annaks need read vea)
# indeksid = np.array(d100.index)
# np.random.shuffle(indeksid)
# d100_segiaetud = d100.loc[indeksid,:]
# osa_suurus = len(d100_segiaetud)//3
# d100_treen = d100_segiaetud[:osa_suurus]
# d100_valid = d100_segiaetud[osa_suurus:(2*osa_suurus)]
# d100_test = d100_segiaetud[(2*osa_suurus):]

In [ ]:
# Lahendus palun siia (taas võib soovi korral koodivälju juurde teha ja teksti juurde kirjutada).

<font color='purple'>**(e) Vahetada treening- ja valideerimisandmestike rollid: see, mis oli enne treeningandmestik, on nüüd valideerimisandmestik, ja vastupidi. Tee samasugune joonis nagu eelmises punktis ning vali parim ribalaius selle uue joonise järgi. Kas parimaks osutus sama või erinev ribalaius? Seejuures salvestage NLL tulemused erinevatesse muutujatesse kui eelmises alapunktis, sest neid läheb järgmises alapunktis vaja.**</font>
    
<font color='purple'>**OODATUD TULEMUS: Joonis, parim ribalaius ja selgitus, kas see oli sama mis eelmises punktis (1 lause).**</font>

In [ ]:
# Lahendus

<font color='purple'>**(f) Vali parim ribalaius 2-osalise ristvalideeringuga, kasutades kahe eelmise alapunkti tulemusi. Tee joonis, kus on 2 osa peale keskmistatud treening-NLL ja 2 osa peale keskmistatud valideering-NLL erinevate ribalaiuste jaoks. Milline on parim ribalaius ning mis on 2 osa peale keskmistatud treening- ja valideerimis-NLL selle ribalaiuse jaoks?**</font>

<font color='purple'>**OODATUD TULEMUS: Joonis, parim ribalaius ning selle 2 osa peale keskmistatud treening- ja valideerimis-NLL (1-3 lauset).**</font>

In [ ]:
# Lahendus

<font color='purple'>**(g) Hinnata 2-osalise ristvalideeringuga, kui hea on KDE tihedushinnang vaikeparameetritega (ehk jättes `bw_method` parameetri üldse panemata). Selleks keskmistada mõlema osa valideering-NLL. Kas tulemus on parem või halvem kui parimal ribalaiusel eelmises alapunktis?**</font>
    
<font color='purple'>**OODATUD TULEMUS: Vaikeparameetritega KDE tihedushinnagu valideering-NLL, hinnatuna 2-osalise ristvalideeringuga. Võrdlus, kas see on parem või halvem kui eelmises alapunktis (1 lause).**
</font>

In [ ]:
# Lahendus

<font color='purple'>**(h) Panna kokku treening- ja valideerimisandmestikud üheks andmestikuks ning leida sellel 2 erinevat KDE tihedushinnangut - üks punktis (f) leitud parima ribalaiusega ning teine vaikimisi parameetritega. Mõõta nende sobivust mõõduga NLL testandmetel. Kumb meetod on parem? Kas sama meetod, mis osutus parimaks eelmises alapunktis (g) või teine meetod? Kas see on üllatav või mitte?**</font>

<font color='purple'>**OODATUD TULEMUS: Test-NLL kahe meetodiga, võrdlus kumb on parem ning selgitus kas järjestus on üllatav või mitte (1-3 lauset).**
</font>

In [ ]:
# Lahendus

# 2. ülesanne.  Kliimamudelid Tartu kohta - Kolmogorov-Smirnovi test ja autokorrelatsion (4,5 punkti)

<font color='purple'> **NB! Järgmistes alamülesannetes toimub anomaaliate võrdlus baasperioodi 1880-1979 suhtes**</font>


<font color='purple'>**(a) Samal andmestikul `d` nagu eelmises ülesandes, leida iga aastakümne kohta (1880-1889, 1890-1899, 1900-1909, ..., 2010-2019) keskmine anomaalia (ehk kuude anomaaliate aritmeetiline keskmine). Teha joonis, kus x-teljel on aastakümned ja y-teljel keskmised anomaaliad. Millal algas teie arvates kliimasoojenemine selle joonise järgi? Mitu kraadi on kliima juba soojenenud, Tartu näitel?**</font>
    
<font color='purple'>**OODATUD TULEMUS: Joonis ja teie arvamus (1-3 lauset).**
</font>

In [ ]:
# Lahendus

<font color='purple'>**(b) Tehke 2 joonist, vastavalt anomaaliate histogrammiga kümnendi 1990-1999 kohta ning kümnendi 2010-2019 kohta. Mõlemile joonisele lisage KDE tihedushinnang, mis on sobitatud vaikimisi parameetritega andmestikule `d100` (ehk kõik anomaaliad ajavahemikus 1880-1979). Kas jooniste järgi näib teile pigem, et andmed on KDE mudelile vastavad või mitte (ehk kas sellest KDE mudelist on tõenäoline saada selliseid andmeid või mitte)?**</font>
    
<font color='purple'>**OODATUD TULEMUS: 2 joonist ja selgitus, kas teie silma järgi vastavad andmed mudelile või mitte (1-3 lauset).**
</font>

In [ ]:
# Lahendus

<font color='purple'>**(c) Tehke 2 joonist, vastavalt empiirilise jaotusfunktsiooniga kümnendi 1990-1999 kohta ning kümnendi 2010-2019 kohta. Mõlemile joonisele lisage KDE jaotusfunktsioon, kus KDE on samuti nagu eelmises alapunktis sobitatud vaikimisi parameetritega andmestikule `d100`. Mõlemal joonisel leidke (ehk arvutage andmete järgi) anomaalia väärtus, mille korral on empiirilise jaotusfunktsiooni ning KDE jaotusfunktsiooni erinevus suurim. Märkige mõlemil joonisel vastav koht ära (näiteks suur punkt mõlemil joonel vastavas kohas või vertikaalne joon nende punktide vahel).**</font>

<font color='purple'>**OODATUD TULEMUS: 2 joonist ning mõlemi kohta anomaalia väärtus, mille korral on empiirilise jaotusfunktsiooni ning KDE jaotusfunktsiooni erinevus suurim (kirjutada välja nii anomaalia väärtus kui vastaval kohal jaotusfunktsioonide vaheline erinevus).**</font>
    
<font color='purple'>VIHJED:</font>

<font color='purple'> 
    
* Kui KDE on leitud meetodiga `kde = scipy.stats.gaussian_kde(d100.anomaalia_1880_1979)`, siis jaotusfunktsiooni joonistamiseks võtke `x_vektor` ühtlaste vahedega andmete miinimumi ja maksimumi vahel (vt. ka eelmise ülesande alapunkti 1c) ja seejärel leidke vastavad jaotusfunktsiooni väärtused järgmiselt: `[kde.integrate_box(-np.inf,x) for x in x_vektor]`. </font>

<font color='purple'> 

* Empiirilise jaotusfunktsiooni joonistamiseks võtke `x_vektor = np.sort(d_valitud.anomaalia_1880_1979)` ja vastav `y_vektor = np.linspace(0, 1, len(d_valitud), endpoint=False)`, kus `d_valitud` on sobiv alamhulk andmestikust (näiteks 1990-1999). </font>

<font color='purple'>
    
* Suurima erinevusega koha leidmiseks minge mööda empiirilise jaotusfunktsiooni vektoreid `x_vektor` ja `y_vektor` ning leidke vastavad KDE jaotusfunktsiooni väärtused: `[kde.integrate_box(-np.inf,x) for x in x_vektor]`.
</font>

In [ ]:
# Lahendus

<font color='purple'>**(d) Viige läbi 1-valimiline Kolmogorov-Smirnovi test (`scipy.stats.ks_1samp`) võrdlemaks kümnendi 1990-1999 anomaaliate empiirilist jaotust aastasaja 1880-1979 KDE-jaotusega (vaikimisi parameetritega). Kontrollige, kas statistiku väärtus on enam-vähem sama, mis eelmises alapunktis leitud suurim erinevus. Seejärel tehke sama võrdlemaks kümnendit 2010-2019 aastasajaga 1880-1979. Kas leitud p-väärtused lükkavad ümber väite et kümnendi 1990-1999 (või 2010-2019) anomaaliad vastavad aastasaja 1880-1979 mudelile? Mida sellest järeldada kliimasoojenemise kohta?**</font>
    
<font color='purple'>**OODATUD TULEMUS: Kolmogorov-Smirnovi testi tulemused, statistiku väärtuse kontroll eelmise alapunktiga, p-väärtuste tõlgendamine ja järeldused (3-5 lauset).** </font>
    
<font color='purple'>VIHJE:</font>

<font color='purple'>
    
* `scipy.stats.ks_1samp` teiseks parameetriks on vaja panna KDE jaotusfunktsioon. Selleks sobib `kde_cdf`, mille saab eelnevalt defineerida nii: `kde_cdf = lambda x: np.array([kde.integrate_box(-np.inf,x) for x in x_vektor])`, kus `kde` on funktsiooniga `scipy.stats.gaussian_kde` saadud sobitamise tulemus.
</font>

In [ ]:
# Lahendus

<font color='purple'>**(e) Tehke 2 joonist, kus ühel on kümnendi 1990-1999 ja teisel kümnendi 2010-2019 empiiriline jaotusfunktsioon. Võrdluseks lisada mõlemale joonisele aastasaja 1880-1979 empiiriline jaotusfunktsioon. Viige läbi 2-valimiline Kolmogorov-Smirnovi test (`scipy.stats.ks_2samp`) võrdlemaks kümnendi 1990-1999 anomaaliate empiirilist jaotust aastasaja 1880-1979 empiirilise jaotusega. Seejärel tehke sama võrdlemaks kümnendit 2010-2019 aastasajaga 1880-1979. Kas leitud p-väärtused erinevad oluliselt võrreldes eelmises alapunktis tehtud 1-valimilise Kolmogorov-Smirnovi testiga? Miks erinevad või ei erine?**</font>
    
<font color='purple'>**OODATUD TULEMUS: Kolmogorov-Smirnovi testi tulemused, p-väärtuste võrdlus ja põhjendus (1-2 lauset).**
</font>

In [ ]:
# Lahendus

<font color='purple'>**(f) On alust arvata, et kahe järjestikuse kuu anomaaliad on korreleeritud. See tekitab olukorra, kus järjestikuseid anomaaliaid ei saa vaadelda täielikult sõltumatutena ning seetõttu ei kehti Kolmogorov-Smirnovi testi eeldus, mille järgi peavad valimi elemendid olema sõltumatud ja sama jaotusega ehk s.s.j. (ingl.k. 'independent and identically distributed' ehk 'i.i.d.'). Sõltuvuste uurimiseks arvutage andmestikus `d100` autokorrelatsioonid nihkega 1 kuud, 2 kuud, ..., 36 kuud. Seda saab teha funktsiooniga `d100.anomaalia_1880_1979.autocorr(...)` (eeldab, et andmestikus on anomaaliad ajalises järjekorras). Teha joonis, kus x-teljel on nihe kuudes ja y-teljel vastav autokorrelatsioon. Oletame, et autokorrelatsioonid, mida näeme rohkem kui 12 kuu puhul on pigem juhuslikku laadi - mitme kuu kaugusele ulatuvad sel juhul olulised korrelatsioonid? (otsustada enda parima äranägemise järgi jooniselt).**</font>
    
<font color='purple'>**OODATUD TULEMUS: Joonis ning selgitus selle kohta kui kaugele ulatuvad olulised autokorrelatsioonid (1-3 lauset).**
</font>

In [ ]:
# Lahendus

<font color='purple'>**(g) Et Kolmogorov-Smirnovi testi rakendamiseks andmed enam-vähem sõltumatud oleks, agregeerige andmed aastakaupa, arvutades iga aasta kohta keskmise anomaalia. Algsetel andmetel saab seda näiteks teha nii: `d_aastakaupa = d.groupby('aasta').mean()`. Mis oli kõige hiljutisem aasta, kus aasta keskmine oli alla aastasaja 1880-1979 keskmise?**</font>

<font color='purple'>**Teha aastakaupa aastate 1880-1979 andmetel samasugune autokorrelatsiooni joonis nagu eelmises alapunktis, kuid nihkega 1 aasta, 2 aastat, ..., 30 aastat. On alust arvata, et need autokorrelatsioonid on kõik enam-vähem juhuslikku laadi.**</font>
    
<font color='purple'>**Viige läbi 2-valimiline Kolmogorov-Smirnovi test (`scipy.stats.ks_2samp`) võrdlemaks kümnendi 1990-1999 aastakaupa keskmiste anomaaliate empiirilist jaotust aastasaja 1880-1979 aastakaupa keskmiste anomaaliate empiirilise jaotusega.  Seejärel tehke sama võrdlemaks kümnendit 2010-2019 aastasajaga 1880-1979. Kas leitud p-väärtused lükkavad ümber väite et kümnendi 1990-1999 (või 2010-2019) anomaaliad vastavad aastasaja 1880-1979 mudelile? Mida sellest järeldada kliimasoojenemise kohta?**</font>
    
<font color='purple'>**OODATUD TULEMUS: Kõige hiljutisem aasta, kus aasta keskmine oli alla aastasaja 1880-1979 keskmise; joonis autokorrelatsioonidega. Kolmogorov-Smirnovi testi tulemused ja nende tõlgendamine ning järeldused (3-5 lauset).**
</font>

In [ ]:
# Lahendus

## <font color='red'>Enne kodutöö esitamist palun lähtesta kernel uuesti ning jooksuta kogu kood algusest peale korra läbi - selleks on vaja menüüst valida `Kernel` -> `Restart and Run All`.</font>

## Mitu tundi tööd kulus kodutöö lahendamiseks?

Palun vasta nii täpselt kui oskad. See ei muuda mingilgi moel punkte, mille kodutöö eest saad. Nii 0,5 tundi kui 24 tundi on sobivad vastused. Loe astronoomilisi tunde (1 tund = 60 minutit), mitte akadeemilisi (45 minutit). Kogutud informatsiooni kasutatakse kodutööde paremaks muutmiseks.
<font color = "red"><b> Palun asenda "X" tundide arvuga ning ära muuda teksti 'tundi' millekski muuks.</b></font>

**<font color='red'>1. ülesanne (asenda X järgmisel väljal oma ajahinnanguga)</font>**

X tundi

**<font color='red'>2. ülesanne (asenda X järgmisel väljal oma ajahinnanguga)</font>**

X tundi

**<font color='red'>KOKKU (asenda X järgmisel väljal oma ajahinnanguga)</font>**

X tundi

**<font color='red'>SUUR TÄNU TEHTUD TÖÖ EEST!</font>**